# COGS 118A - Final Project

## Prediction of Chess Game Outcomes Based on Player Features

# Names

Hopefully your team is at least this good.

- Omri Habot
- Keate Ehrenburg
- Jacob Lamadrid
- Alex Bumbalov

# Abstract

The goal of this project is to examine the effectiveness of different models in predicting the outcome of a chess match. The data used in this project will contain features describing players’ technical skill and experience including the Elo rating of each player, the number of games they have played, the number of wins/losses/draws they have achieved, their age, and the length of the game. These are measured by observing the outcomes of every game in a player’s career. We will use this data to perform feature selection and single out the most relevant features, and then we will train the models and compare their predictive accuracies. Accuracy will be measured by classification error metrics including precision, recall, f1-score, and possibly others, which we will compare across models to observe their relative efficacies.

# Background

Within the world of chess competitions and general play, machine learning and deep learning have been famously applied in such algorithms as Deep Blue, Stockfish, etc. These algorithms historically have aimed to predict the next best move to be executed as well as the win probability at any given position state [1](https://arxiv.org/pdf/2109.11602.pdf). This win probability is what we aim to place our project emphasis upon as in similar works in varying competitive settings, most relevant of which is found in esports win/loss classification based upon player/team rankings and typical movesets among many other features [2](https://arxiv.org/pdf/2108.02799.pdf). The application of these algorithms in returning win probability and outcome prediction has large impacts in the way one chooses to learn chess or the way in which a machine is taught how to play chess. This may also have implications for the way in which new strategies or entire play styles are formed. These implications have already manifested themselves in the competitive playspace as many players look towards Stockfish evaluations for input on their play or for other engines in which competition may aid in their training as “with the help of chess engines, Grandmasters are now able to plan prepare for their games in extreme depth, sometimes memorizing up to 15-20 moves of their openings” [3](https://fluency.mcsaatchi.com/2022/09/01/the-evolution-of-chess-ai/#:~:text=Artificial%20intelligence%20has%20completely%20changed,20%20moves%20of%20their%20openings.). In order to approach the problem of finding an appropriate machine learning prediction method , we are going to need a large amount of chess match data to work with. We have chosen to use the Free Internet Chess Server(FICS) database. The database has been collecting all rated and unrated games played on FICS since October of 2008. With over 200 million games stored, this will provide us with plenty of data to address our goal of outcome classification.

Our models will train similarly to current chess engines, utilizing previous games played in order to gain improve decision making and outcome prediction, as in the models created by the chess.com website, with machine learning algorithms such as Luk.AI which improves its elo with every game played.

# Problem Statement

The problem we will be addressing is determining which machine learning method best predicts the winner of a chess game depending on variables like participating players, player statistics and scores such as their Elo’s, the opening, and psychological factors of the players, among others. We plan to explore this problem by trying three different machine learning algorithms: logistic regression, random forest, and deep neural network. Logistic regression is a solid classification method. While we fear that it may be too simple for this task, we believe it is a simple baseline to compare with other, more complex models. We plan to try random forests because the complex decision boundaries may be sufficient for consistently accurate and generalizable predictions. Finally, we plan on using a deep neural network because it seems to be the baseline method used for very similar problems, and given our large dataset, we hope it will pick up on important nuances that are neglected by logistic regression and random forest. We will use labeled chess game data that denotes the winner of each game as well as the aforementioned variables that describe the game. Each model will be trained using the same (or similar) dataset and evaluated with the same set of classification error metrics such that we can compare their efficacies. These metrics include precision, recall, f1-score, and others.

# Data

We will be using the [Free Internet Chess Server (FICS)](https://www.ficsgames.org/dl/ficsgamesdb_202301_blitz2000_movetimes_278527.pgn.bz2) as our database for analysis. The dataset comes in the Portable Game Notation (PGN) file format, which is a common data format for representing chess games. We will need to convert this into a pandas dataframe for our analysis, so we will write a function that parses through the PGN file as text and builds a dataframe from it. Once the data is parsed, we will simply need to convert each column into the appropriate datatype, so numbers would be int32 or float64, words will be strings, binary features will be boolean, etc.

There are many different types of chess games from which we can choose that would cause nuances in our analysis and results. We selected blitz games because their rapid nature produces more variability in game moves, and therefore outcomes. We hope that this variability gives us more data to work with when predicting outcomes. Our dataset has 25884 complete observations (each representing a chess game) as well as 22 features describing each game (before one-hot encoding). Each observation consists of the moves made throughout the game, the date and time played, the players and their colors, player Elo, match result, and game variant and time control information for each match. One of the most critical features is `playerElo`, a score representing the relative skill levels of players, since this is likely one of the biggest predictors of the outcome. This is represented by an arbitrary number that can indicate a player's relative skill level when compared to other players. Another crucial feature is `moves`, which outlines all the moves made in the game. We can use this sequential data to build more complex models and make more nuanced predictions.

### Data Cleaning and Importing

The database provides chess game data in the PGN file format. In order to convert the data into a dataframe and proceed with our analysis, we wrote scripts to parse our desired variables from PGN files and produce CSV files. Afterwards, we imported the CSV's as dataframes and cleaned them up such that each feature had the proper datatype. 

In [2]:
# Cleaning
import pandas as pd
import os

# import all csv's from a folder and combine them into one dataframe
def combine_data_files(folder_path):
    df_list = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            df_list.append(df)

    return pd.concat(df_list, axis=0, ignore_index=True)

# clean up the datatypes of the imported dataframes
def clean_df_new(df):
    
    resMap = lambda res : 'White' if res == '1-0' else 'Black' if res == '0-1' else 'Draw' if res == "1/2-1/2" else res
    cleanDateTime = lambda d: pd.to_datetime(d)
    cleanComment = lambda x: x if x.find(']') == -1 else x.split("] ")[1]

    datetime_columns = ['Date', 'Time', 'WhiteClock', 'BlackClock']
    float_columns = ['WhiteRD', 'BlackRD', 'AvgEvalOpening', 'AvgEvalMiddle', 'AvgEvalEnd', 'AvgEmtOpening', 'AvgEmtMiddle', 'AvgEmtEnd']
    int_columns = ['FICSGamesDBGameNo', 'WhiteElo', 'BlackElo', 'PlyCount']

    df = df.drop(['Event', 'Site', 'Round', 'WhiteClock', 'BlackClock'], axis=1)

    for column in df.columns:

        if column in float_columns:
            df[column] = df[column].astype(float)
        elif column in int_columns:
            df[column] = df[column].astype(int)
        elif column in datetime_columns:
            df[column] = df[column].apply(cleanDateTime)
        else:
            if column == 'Result':
                df[column] = df[column].apply(resMap)
            if column == 'ResultComment':
                df[column] = df[column].apply(cleanComment)
            df[column] = df[column].astype('category')

    df['WhiteIsComp'] = df['WhiteIsComp'] == 'Yes'
    df['BlackIsComp'] = df['BlackIsComp'] == 'Yes'

    return df

In [9]:
combined_df = combine_data_files('data-by-month')
cleaned = clean_df_new(combined_df).dropna()
cleaned.shape

(25884, 22)

After cleaning the datatypes, we performed the last processing step: applying one-hot encoding to categorical features and converting redundant numerical features into averages.

In [3]:
# additional processing: perform one-hot encoding and take average of given numerical columns
def construct_df_train(df):
    # Define columns to take the average of and to one-hot encode
    cols_to_avg = ['WhiteRD', 'BlackRD', 'AvgEvalOpening', 'AvgEvalMiddle', 'AvgEvalEnd', 'AvgEmtOpening', 'AvgEmtMiddle', 'AvgEmtEnd', 'PlyCount']
    cols_to_onehot = ['ECO'] # 'White', 'Black',  <--- For if we decide to train using player UN for black and white as a features
    ground_truth_labels = ['Result', 'ResultComment']

    # Define dictionary to map column names to letters for one-hot encoding
    col_name_to_letter = {col_name: chr(65+i) for i, col_name in enumerate(ground_truth_labels+cols_to_onehot)}

    # Define column names for player-wise averages
    new_cols_white = ['WhiteAvgRD', 'WhiteAvgEvalForOpenings', 'WhiteAvgEvalForMiddlegames', 'WhiteAvgEvalForEndgames', 'WhiteAvgEmtForOpenings', 'WhiteAvgEmtForMiddlegames', 'WhiteAvgEmtForEndgames', 'WhiteAvgPlyCount']
    new_cols_black = ['BlackAvgRD', 'BlackAvgEvalForOpenings', 'BlackAvgEvalForMiddlegames', 'BlackAvgEvalForEndgames', 'BlackAvgEmtForOpenings', 'BlackAvgEmtForMiddlegames', 'BlackAvgEmtForEndgames', 'BlackAvgPlyCount']

    # Compute player-wise averages
    dataAvgsByUserWhite = {p: df[df['White'] == p][cols_to_avg+cols_to_onehot].mean(numeric_only=True) for p in df['White'].unique()}
    dataAvgsByUserBlack = {p: df[df['Black'] == p][cols_to_avg+cols_to_onehot].mean(numeric_only=True) for p in df['Black'].unique()}

    # Define empty dataframe to hold the new data
    df_new = pd.DataFrame(columns = ground_truth_labels+cols_to_onehot+new_cols_white+new_cols_black)

    # Iterate over each row in the original dataframe and create a new row in the output dataframe with the necessary data
    for row in df.itertuples():
        white_avg_values = dataAvgsByUserWhite.get(row.White, pd.Series([0]*len(cols_to_avg+cols_to_onehot)))
        black_avg_values = dataAvgsByUserBlack.get(row.Black, pd.Series([0]*len(cols_to_avg+cols_to_onehot)))

        white_new_cols = [white_avg_values[0], white_avg_values[2], white_avg_values[3], white_avg_values[4], white_avg_values[5], white_avg_values[6], white_avg_values[7], white_avg_values[8]]
        black_new_cols = [black_avg_values[1], black_avg_values[2], black_avg_values[3], black_avg_values[4], black_avg_values[5], black_avg_values[6], black_avg_values[7], black_avg_values[8]]

        new_row = tuple(getattr(row, label) for label in ground_truth_labels+cols_to_onehot) + tuple(white_new_cols) + tuple(black_new_cols)

        df_new.loc[len(df_new)] = new_row

    # One-hot encode the ground truth labels and cols_to_onehot
    df_new = pd.get_dummies(df_new, columns = ground_truth_labels+cols_to_onehot, prefix = list(col_name_to_letter.values()))

    # Map the column labels to their corresponding uppercase letters
    df_new.columns = df_new.columns.map(lambda x: col_name_to_letter.get(x, x))

    return df_new

In [8]:
df_train = construct_df_train(cleaned)
print('Number of observations: ' + str(df_train.shape[0]))
print('Number of features (after one-hot encoding): ' + str(df_train.shape[1]))
df_train.head()

Number of observations: 25884
Number of features (after one-hot encoding): 517


,WhiteAvgRD,WhiteAvgEvalForOpenings,WhiteAvgEvalForMiddlegames,WhiteAvgEvalForEndgames,WhiteAvgEmtForOpenings,WhiteAvgEmtForMiddlegames,WhiteAvgEmtForEndgames,WhiteAvgPlyCount,BlackAvgRD,BlackAvgEvalForOpenings,...,C_E87,C_E88,C_E90,C_E91,C_E92,C_E93,C_E94,C_E95,C_E96,C_E97
0,29.348602,23.614612,16.099574,28.836774,1.897313,2.309180,1.631107,160.004658,35.060330,-4.545055,...,0,0,0,0,0,0,0,0,0,0
1,34.851760,54.018170,128.992509,265.426934,1.435165,2.490632,2.255974,101.928292,30.245794,24.857252,...,0,0,0,0,0,0,0,0,0,0
2,34.075718,45.014569,104.508098,195.642712,2.104136,3.775255,3.686790,92.955514,30.245794,24.857252,...,0,0,0,0,0,0,0,0,0,0
3,49.572727,60.490440,140.078384,192.691169,1.725352,3.174036,2.597224,89.421488,34.200000,52.580662,...,0,0,0,0,0,0,0,0,0,0
4,22.756332,8.380797,-52.832145,-53.019966,0.547576,1.102669,1.044522,81.209607,33.835407,29.418740,...,0,0,0,0,0,0,0,0,0,0


# Proposed Solution

In this project we aim to evaluate the effectiveness of three different machine learning implementations at classifying the outcomes of FIDE titled chess games from unlabeled match data. Each model (Logisitc Regression, Random Forests, and the Deep NN) will be trained on a curated sample database consisting of several derived features representing characteristics of the games pertinent to the games’ outcomes. We chose logistic regression as our first model because it is a solid classification method. While we fear that it may be too simple for this task, we believe it is a good, simple baseline to compare with more complex models. We also plan to try random forests because the complex decision boundaries may be sufficient for consistently accurate and generalizable predictions. Finally, we will use a deep neural network because it seems to be the baseline method used for very similar problems regarding chess game outcomes, and given our large dataset, we hope it will pick up on important nuances that are neglected by logistic regression and random forest. 

For this we will need to use a variety of tools for accessing and manipulating the data to generate these special features. Pandas and numpy will be instrumental in this preprocessing stage. Further imports will be required for sklearn and matplot lib for training/validation and analysis, as well as an additional library called pychess will be necessary for processing chess engine ratings and to help format game data for preprocessing. We will be using built in methods for training and validating data provided by sklearn. To measure the effectiveness of our model we will take Sayon Bhattacharjee’s LSTM solution published in the _Towards Data Science_ article found [here](https://towardsdatascience.com/predicting-professional-players-chess-moves-with-deep-learning-9de6e305109e). Sayon's LTSM solution uses two convolutional neural networks; one "predicts" where a particular piece moved from, and the other predicts where a piece will move. These networks are combined as a time series to model a classification problem with three possible output categories corresponding with the outcome: black wins, white wins, or there is a draw. The effectiveness of this model can simply be measured by classification accuracy using metrics discussed in the following section.

# Evaluation Metrics

In the evaluation of binary chess outcome classification, simply implementing a confusion matrix of our algorithm’s results is most appropriate for visualizing the performance. In this situation, there is equal cost associated with both false positive rates and false negative rates, and there is equal cost associated with true positives and true negative rates. Consequently, we care about both the recall and precision of our models equally. This suggests that an f1-score is one primary metric we should use, since it is equally sensitive to both recall and precision. For example, predicting that Player 1 wins when they actually win is equally as important as being ‘accurate’ when predicting that Player 1 will win; the cost of a false positive is no different than the cost of a false negative. We may also explore some other classification error metrics such as likelihood ratios.

$$precision = \frac{TP}{TP + FP}$$
$$recall = \frac{TP}{TP + FN}$$
$$F1-Score = \frac{2*precision*recall}{precision + recall}$$

# Results
# __NEEDS WORK__
You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result
# __NEEDS WORK__
OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations
# __NEEDS WORK__
Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

In the case of data privacy, we see no major concerns with our data as it is publicly available FIDE match information that is made public anytime a chess player agrees to a FIDE match. The biggest ethical concern that we could potentially see with the production of any well-functioning model for predicting FIDE titled chess matches would be the use of the model for gambling purposes. Although we don't foresee the prediction accuracy of our models being able to give a definitive answer to who will win a match, a good model could lead to an increase in chess match gambling which comes with its own host of harmful consequences. Additionally, we can highlight the concern that a prediction model could affect the way that players go about their matches if they know the predicted outcome of a game given specific features. This could be seen as an unfair advantage. The model, if then re-trained with new data, could also perform much more poorly as outcomes would now be affected by prior knowledge of the potential outcome of a game which would complicate further predictions.

### Conclusion
# __NEEDS WORK__
Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.